In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'car-plate-detection:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F686454%2F1203932%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241011%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241011T143642Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4539fba9a4b9cc8d06a61ae50b848d87b6f5dfdb15ba304f8125561ce625bd4df47dc1deb2d02025f317249abdff840890a2212b048a6671aafca2db0464551ec3c26c0fd3f148bcdb09d7fea13250f44dc65dcdae1b780e3f3343425afb4c43cc1393e030a5acb160a55973b2f4054b5e55aac108f333b61aeb97ac05d41a1920d98e5b0fe525d7a839830506fd4ee8386cddbca8137531a37e9cb02d4901f16c4703123c4a9925c3714ea7f6149285f1b7569a1714a071869230196b90c028c81d171c2204a8e0da24e789eb17c5df7aab34549503251e898fc72efd33d5c208feed52b6529da4175b9138d112bb4dc9e41e11fedde4dfee419db685e44e32'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 212910636 bytes downloaded
Downloaded and uncompressed: car-plate-detection
Data source import complete.


In [2]:
##Start Code Here
import pandas as pd
import xml.etree.ElementTree as xet
from glob import glob

In [3]:
path = glob('/kaggle/input/car-plate-detection/annotations/*.xml')
path

['/kaggle/input/car-plate-detection/annotations/Cars379.xml',
 '/kaggle/input/car-plate-detection/annotations/Cars88.xml',
 '/kaggle/input/car-plate-detection/annotations/Cars401.xml',
 '/kaggle/input/car-plate-detection/annotations/Cars265.xml',
 '/kaggle/input/car-plate-detection/annotations/Cars212.xml',
 '/kaggle/input/car-plate-detection/annotations/Cars305.xml',
 '/kaggle/input/car-plate-detection/annotations/Cars135.xml',
 '/kaggle/input/car-plate-detection/annotations/Cars302.xml',
 '/kaggle/input/car-plate-detection/annotations/Cars17.xml',
 '/kaggle/input/car-plate-detection/annotations/Cars281.xml',
 '/kaggle/input/car-plate-detection/annotations/Cars275.xml',
 '/kaggle/input/car-plate-detection/annotations/Cars312.xml',
 '/kaggle/input/car-plate-detection/annotations/Cars280.xml',
 '/kaggle/input/car-plate-detection/annotations/Cars99.xml',
 '/kaggle/input/car-plate-detection/annotations/Cars370.xml',
 '/kaggle/input/car-plate-detection/annotations/Cars199.xml',
 '/kaggle/i

In [4]:
labels_dict = dict(filepath=[],xmin=[], xmax =[], ymin = [], ymax=[])
for filename in path:
    info = xet.parse(filename)
    root = info.getroot()
    ## Check file xml to see the directory
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)
    print(xmin, xmax, ymin, ymax)
    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

197 351 197 245
168 242 148 171
158 238 223 240
87 216 166 212
105 290 94 141
361 434 167 183
260 319 184 211
222 425 98 149
306 351 188 205
151 243 169 183
99 162 150 167
111 518 66 246
191 213 185 195
70 262 198 271
285 425 249 276
59 310 88 158
148 262 180 213
36 62 175 186
139 249 282 310
159 250 178 200
109 159 182 206
177 216 139 159
102 125 122 135
313 371 135 150
276 396 186 273
274 350 257 297
212 245 182 203
162 211 179 188
291 410 167 190
253 284 163 177
189 259 55 78
181 269 264 289
168 240 196 231
91 164 110 138
166 206 141 154
116 273 156 188
164 240 202 220
116 525 55 262
122 277 82 117
64 133 160 180
143 170 263 271
191 255 165 192
270 321 162 195
148 305 113 178
135 218 134 158
189 384 161 215
262 318 141 160
54 120 135 178
160 246 149 171
77 101 204 225
55 343 82 147
126 241 142 199
164 245 104 119
285 374 185 226
146 246 114 163
166 224 142 176
114 229 79 101
167 240 202 221
126 173 180 205
42 202 131 243
297 316 193 205
166 273 225 246
112 306 65 173
176 326 254 284

In [5]:
labels_dict

{'filepath': ['/kaggle/input/car-plate-detection/annotations/Cars379.xml',
  '/kaggle/input/car-plate-detection/annotations/Cars88.xml',
  '/kaggle/input/car-plate-detection/annotations/Cars401.xml',
  '/kaggle/input/car-plate-detection/annotations/Cars265.xml',
  '/kaggle/input/car-plate-detection/annotations/Cars212.xml',
  '/kaggle/input/car-plate-detection/annotations/Cars305.xml',
  '/kaggle/input/car-plate-detection/annotations/Cars135.xml',
  '/kaggle/input/car-plate-detection/annotations/Cars302.xml',
  '/kaggle/input/car-plate-detection/annotations/Cars17.xml',
  '/kaggle/input/car-plate-detection/annotations/Cars281.xml',
  '/kaggle/input/car-plate-detection/annotations/Cars275.xml',
  '/kaggle/input/car-plate-detection/annotations/Cars312.xml',
  '/kaggle/input/car-plate-detection/annotations/Cars280.xml',
  '/kaggle/input/car-plate-detection/annotations/Cars99.xml',
  '/kaggle/input/car-plate-detection/annotations/Cars370.xml',
  '/kaggle/input/car-plate-detection/annotatio

In [6]:
df = pd.DataFrame(labels_dict)
df

,filepath,xmin,xmax,ymin,ymax
0,/kaggle/input/car-plate-detection/annotations/...,197,351,197,245
1,/kaggle/input/car-plate-detection/annotations/...,168,242,148,171
2,/kaggle/input/car-plate-detection/annotations/...,158,238,223,240
3,/kaggle/input/car-plate-detection/annotations/...,87,216,166,212
4,/kaggle/input/car-plate-detection/annotations/...,105,290,94,141
...,...,...,...,...,...
428,/kaggle/input/car-plate-detection/annotations/...,146,260,187,213
429,/kaggle/input/car-plate-detection/annotations/...,282,422,236,283
430,/kaggle/input/car-plate-detection/annotations/...,319,478,165,204
431,/kaggle/input/car-plate-detection/annotations/...,90,273,145,196


In [7]:
df.to_csv('labels.csv', index=False)